In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle


In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Preprocess the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [5]:
#Encode categeorical Variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [6]:
#One-hot encode 'Geography'
onehot_encoded_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoded_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoded_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
#Combine one-hot encoded columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
#split data into features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [9]:
##split the data in training and testing

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [10]:
#Scale the feature

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [11]:
#Save the encoder and scaler for later use

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [12]:
## Train ANN Regresion Problem

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
#Build ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), 
    Dense(32,activation='relu'),
    Dense(1) #Output layer for regression
])

#compile model

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
##set up tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [15]:
 ##set up EarlyStopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [16]:
#Training model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorboard_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 4ms/step - loss: 100376.9688 - mae: 100376.9688 - val_loss: 98512.8125 - val_mae: 98512.8125
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99612.5625 - mae: 99612.5625 - val_loss: 96964.7266 - val_mae: 96964.7266
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 96923.5234 - mae: 96923.5234 - val_loss: 93007.0000 - val_mae: 93007.0000
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 91648.8750 - mae: 91648.8750 - val_loss: 86439.7891 - val_mae: 86439.7891
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 83936.1484 - mae: 83936.1484 - val_loss: 77886.3750 - val_mae: 77886.3750
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 74810.4844 - mae: 74810.4844 - val_loss: 68890.7969 - val_mae: 68890.7969
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 65954.5156 - mae: 65954.515

In [17]:
%load_ext tensorboard

In [18]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 8156), started 3:08:00 ago. (Use '!kill 8156' to kill it.)

In [19]:
#Evaluate model test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 2ms/step - loss: 50278.8281 - mae: 50278.8281
Test MAE : 50278.828125


In [20]:
model.save('regression_model.h5')

c:\Users\Hp\anaconda3\envs\myenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
